In [1]:
from pathlib import Path
import numpy as np, pydicom, csv
from pydicom.pixel_data_handlers.util import apply_modality_lut, apply_voi_lut
from PIL import Image
from scipy.ndimage import zoom  # 等方リサンプリング用（線形補間）

# ====== ここだけご自身の環境に合わせてください ======
SRC_DIR = "/Users/tkimura/Desktop/DICOM-A/ScalarVolume_37"   # DICOMのフォルダ
OUT_DIR = "/Users/tkimura/Desktop/DICOM-A/out_iso_views"     # 出力先フォルダ
FMT     = "png"     # "png" または "jpg"
WL, WW  = 40, 400   # 軟部窓（肺:-600/1500, 骨:300/1500）
AUTO    = False     # True: 自動WL/WW(0.5〜99.5%)
# 等方化（mm）。None の場合は "元の px,py,pz の最小値" を採用（= いちばん細かい解像度に合わせる）
ISO_SPACING_MM = None
INTERP_ORDER   = 1   # 0=ニアレスト, 1=線形, 3=3次
SAVE_AXIAL    = True
SAVE_CORONAL  = True
SAVE_SAGITTAL = True
# 必要に応じて見た目調整（左右/上下の反転）
FLIP_AXIAL_LR = False; FLIP_AXIAL_UD = False
FLIP_COR_LR   = False; FLIP_COR_UD   = False
FLIP_SAG_LR   = False; FLIP_SAG_UD   = False
# ================================================

src = Path(SRC_DIR); out_root = Path(OUT_DIR)
(out_root).mkdir(parents=True, exist_ok=True)

def _is_dicom(p: Path):
    try:
        with open(p, "rb") as f:
            pre = f.read(132)
            return pre[128:132] == b"DICM"
    except Exception:
        return False

def _safe_spacing(ds):
    # X=列方向, Y=行方向, Z=スライス方向
    px = py = 1.0
    ps = getattr(ds, "PixelSpacing", None)
    if ps is not None and len(ps) >= 2:
        try:
            px, py = float(ps[0]), float(ps[1])
        except Exception:
            px = py = 1.0
    pz = getattr(ds, "SpacingBetweenSlices", None)
    if pz is not None:
        try:
            pz = float(pz)
        except Exception:
            pz = None
    if pz is None:
        try:
            pz = float(getattr(ds, "SliceThickness"))
        except Exception:
            pz = px
    return float(px), float(py), float(pz)

# 1) DICOM収集 & 最大スライスのシリーズを選択
series = {}
for p in src.rglob("*"):
    if p.is_file() and _is_dicom(p):
        try:
            ds = pydicom.dcmread(str(p), stop_before_pixels=True, force=True)
            key = (getattr(ds,"StudyInstanceUID",None), getattr(ds,"SeriesInstanceUID",None))
            if key[1]:
                series.setdefault(key, []).append(p)
        except Exception:
            pass
if not series:
    raise RuntimeError(f"No DICOM series found under: {src}")
key = max(series, key=lambda k: len(series[k]))
files = series[key]

# 2) 並び順（ImagePositionPatientのZ優先、なければInstanceNumber）
metas = []
for p in files:
    ds = pydicom.dcmread(str(p), stop_before_pixels=True, force=True)
    inst = int(getattr(ds,"InstanceNumber", 0))
    ipp  = getattr(ds,"ImagePositionPatient", None)
    z = float(ipp[2]) if (ipp and len(ipp)==3) else None
    metas.append((p, z, inst))
if any(m[1] is not None for m in metas):
    metas.sort(key=lambda x: (x[1] is None, x[1]))
else:
    metas.sort(key=lambda x: x[2])
files = [m[0] for m in metas]

# 3) 読み込み & HU変換
ds0 = pydicom.dcmread(str(files[0]), force=True)
px, py, pz = _safe_spacing(ds0)   # mm
Y, X = int(ds0.Rows), int(ds0.Columns)
Z    = len(files)

vol = np.zeros((Z, Y, X), dtype=np.float32)
for i, p in enumerate(files):
    ds = pydicom.dcmread(str(p), force=True)
    arr = ds.pixel_array
    try:
        arr = apply_modality_lut(arr, ds)  # HU相当に
    except Exception:
        arr = arr.astype(np.float32)
    else:
        arr = arr.astype(np.float32)
    if AUTO and hasattr(ds, "VOILUTSequence"):
        arr = apply_voi_lut(arr, ds).astype(np.float32)  # 自動窓時の補助
    vol[i] = arr

# 4) 等方化（Z,Y,X の順にスケール指定）
if ISO_SPACING_MM is None:
    iso = min(px, py, pz)  # いちばん細かい解像度に合わせる
else:
    iso = float(ISO_SPACING_MM)

# zoom の拡大率： new_dim = old_dim * (old_spacing / new_spacing)
fz = pz / iso
fy = py / iso
fx = px / iso
newZ = max(1, int(round(Z * fz)))
newY = max(1, int(round(Y * fy)))
newX = max(1, int(round(X * fx)))

# メモ：order=1(線形) が速さと品質のバランス良
vol_iso = zoom(vol, zoom=(fz, fy, fx), order=INTERP_ORDER, mode='nearest', grid_mode=True)
# 念のため整数サイズへ合わせ（zoomは端で誤差が出ることがある）
vol_iso = vol_iso[:newZ, :newY, :newX]

# 5) ウィンドウ処理（WL/WW または 自動）
if AUTO or WL is None or WW is None:
    Z2, Y2, X2 = vol_iso.shape
    samp = vol_iso[::max(Z2//16,1), ::max(Y2//16,1), ::max(X2//16,1)].ravel()
    vmin, vmax = np.percentile(samp, 0.5), np.percentile(samp, 99.5)
else:
    vmin, vmax = WL - WW/2.0, WL + WW/2.0

vol01 = np.clip((vol_iso - vmin) / max(1e-6, (vmax - vmin)), 0.0, 1.0)

def _save_stack(stack3d, out_dir: Path, flip_lr=False, flip_ud=False):
    out_dir.mkdir(parents=True, exist_ok=True)
    num_width = max(3, len(str(stack3d.shape[0])))
    for i, slice2d in enumerate(stack3d, start=1):  # 001始まり
        img = (slice2d*255.0 + 0.5).astype(np.uint8)
        if flip_lr: img = np.fliplr(img)
        if flip_ud: img = np.flipud(img)
        fname = f"{i:0{num_width}d}"
        if FMT.lower() in ("jpg","jpeg"):
            Image.fromarray(img, mode="L").save(out_dir / f"{fname}.jpg", quality=95, subsampling=0)
        else:
            Image.fromarray(img, mode="L").save(out_dir / f"{fname}.png")

# 6) 3面出力（等方化後の体積 vol01 を使用）
# 等方後のボクセルサイズは (iso, iso, iso) mm
if SAVE_AXIAL:
    _save_stack(vol01, out_root / "axial", flip_lr=FLIP_AXIAL_LR, flip_ud=FLIP_AXIAL_UD)

if SAVE_CORONAL:
    cor = np.transpose(vol01, (1,0,2))   # (Y, Z, X) → 画像は (Z, X)
    _save_stack(cor, out_root / "coronal", flip_lr=FLIP_COR_LR, flip_ud=FLIP_COR_UD)

if SAVE_SAGITTAL:
    sag = np.transpose(vol01, (2,0,1))   # (X, Z, Y) → 画像は (Z, Y)
    _save_stack(sag, out_root / "sagittal", flip_lr=FLIP_SAG_LR, flip_ud=FLIP_SAG_UD)

# 7) メタデータ（Blender/UE の配置用）
with open(out_root / "metadata.csv", "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["SeriesInstanceUID", key[1]])
    w.writerow(["Original_Shape_ZYX", Z, Y, X])
    w.writerow(["Original_Spacing_mm_XYZ", px, py, pz])
    w.writerow(["Isotropic_Spacing_mm", iso])
    Z2, Y2, X2 = vol01.shape
    w.writerow(["Isotropic_Shape_ZYX", Z2, Y2, X2])
    # 3面の1ピクセルの物理寸法（等方化後はすべて iso mm）
    # Axial:    pixel=(iso, iso), slice_step=iso
    # Coronal:  pixel=(iso, iso), slice_step=iso
    # Sagittal: pixel=(iso, iso), slice_step=iso
    w.writerow(["Axial_pixel_mm_(H,W)_&slice_step_mm",   (iso, iso), iso])
    w.writerow(["Coronal_pixel_mm_(H,W)_&slice_step_mm", (iso, iso), iso])
    w.writerow(["Sagittal_pixel_mm_(H,W)_&slice_step_mm",(iso, iso), iso])

print("✅ Done.")
print("Series:", key[1])
print("Original  (Z,Y,X):", (Z, Y, X), "Spacing(mm XYZ):", (px, py, pz))
print("Isotropic (Z,Y,X):", vol01.shape, "Spacing(mm):", iso)
print("Saved to:", str(out_root))


✅ Done.
Series: 1.2.826.0.1.3680043.8.498.11269605138760728492961351834372235563
Original  (Z,Y,X): (143, 512, 512) Spacing(mm XYZ): (0.683, 0.683, 2.0)
Isotropic (Z,Y,X): (419, 512, 512) Spacing(mm): 0.683
Saved to: /Users/tkimura/Desktop/DICOM-A/out_iso_views
